In [ ]:
!pip install -q transformers
!apt-get install portaudio19-dev python-all-dev
!apt install libasound-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg libav-tools
!pip3 install SpeechRecognition
!pip3 install pyaudio
!pip3 install gTTS
!pip3 install pydub

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-all-dev is already the newest version (2.7.15~rc1-1).
portaudio19-dev is already the newest version (19.6.0-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libasound2-dev' instead of 'libasound-dev'
Package libav-tools is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  ffmpeg

E: Package 'libav-tools' has no installation candidate
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple

In [ ]:
from transformers import pipeline
from IPython.display import Javascript, Audio
from google.colab import output
from base64 import b64decode
import speech_recognition as spreg
import os
from pydub import AudioSegment
from gtts import gTTS

In [ ]:
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)

Downloading:   0%|          | 0.00/768 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.webm','wb') as f:
    f.write(b)
  a = AudioSegment.from_file('audio.webm')
  a.export('audio.wav', format='wav')
  return 'audio.webm'  # or webm ?

In [ ]:
rec_time = 5
record(rec_time)
sound_file = 'audio.wav'
recog = spreg.Recognizer()
with spreg.AudioFile(sound_file) as source:
   speech = recog.record(source)
   try:
      text = recog.recognize_google(speech)
      audio = gTTS(text=text, lang='en', slow=False)
      audio.save("tts.mp3")
      print(text)
   except spreg.UnknownValueError:
      print('Unable to recognize the audio')
   except spreg.RequestError as e: 
      print("Request error from Google Speech Recognition service; {}".format(e))

prediction = classifier(text)
print(prediction)
emo = []
for p in prediction[0]:
    if p['score'] > 0.2:
        emo.append(p['label'])

if emo:
    response = "I detect " + ", ".join(emo[:-2] + [" and ".join(emo[-2:])]) + "."
else:
    response = "I'm sorry, but I'm not sure how you're feeling."

audio = gTTS(text=response, lang='en')
audio.save("tts.mp3")
Audio(filename="tts.mp3", autoplay=True)

<IPython.core.display.Javascript object>